# Data Collection Strategy

* I decided to go with webscrapping of the website magicbricks.com to get information of different realestate in mumbai.
* The collection of the data is done using Selenium which uses chromedriver to access my chrome browser. It will then scroll down on the page to get the next set of records.
* This method of collection was implemented because URLlib was being denied access to the website and accessing the website through a browser was the only way.
* After collecting the data I have created a csv file for the same and stored it locally on my machine.

##### Challenges faced

* Execution time for scrapping from selenium is acceptable. For extracting 210 records it takes 43 seconds.
* Extraction of BHK,advertiser and price was easy since it had its own seperate class respectively.
* Other parameters came under a single class which was a card summary on the page for each property.
* Finding all classes for each records takes some time. However converting these records into a numpy array is a better strategy since arrays are faster and occupy lesser memory space. 
* Posted Date was present under another class name so I was not able to use the same function to extract the card parameters. I also had to tackle certain records that had yesterday and today instead of the date. Used date function to tackle this problem.
* There were some records that had "for sale" before location and some had "for sale in" before location. Luckily there were only these 2 versions of this value in each card_record. So I had to write a seperate loop for location to check for both "for sale" and "for sale in".
* Lastly I wrote code that would concat any new records to the previous csv created so that my database keeps increasing.
* Since I have written code for concatinating, we can automate this script to run everyweek or everyday and capture 50-100 records per day to get latest records added into are dataset.
* I would like to store this information in an S3 bucket on amazon aws or any other cloud based solution that is of least cost.

### Import libraries

In [1]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
import time
from datetime import date
from datetime import timedelta
import os
import warnings
warnings.simplefilter('ignore')

### URL from where we are fetching the data

In [2]:
loc = 'Mumbai'
# url = 'https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment&cityName='+loc
url = 'https://www.magicbricks.com/property-for-sale/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment&cityName=Mumbai&BudgetMin=30-Lacs&BudgetMax=50-Lacs'

### Scrapping the data using Selenium library and chromedriver.

In [3]:
start = time.time()
driver = webdriver.Chrome('Drivers/chromedriver.exe')

driver.get(url)
# driver.maximize_window()
for i in range(15): #Running the loop for 50 instances. We roughly get 10 records for each instance but this varies due to the page structure.
    
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)") #This code is responsible for scrolling the page down so that more records are populated.
    time.sleep(0.6) #A sleep time is given to prevent magic bricks from locking us out due to server overload.
end = time.time()
print("Execution time:",end-start) #To calculate time for execution

Execution time: 25.018433332443237


In [4]:
start = time.time()
page_html = bs(driver.page_source,"html.parser") #passing the page_source from driver object into an html parser.
end = time.time()
print("Execution time:",end-start)

Execution time: 29.02517557144165


##### First level of extraction of data from html

In [5]:
s1 = time.time()

#Extracting values from the html using findAll and class name. For example in the 1st case inside span we have that class name mentioned on the page.
price_html = page_html.findAll('div',{'class':'m-srp-card__price'})
date_created = page_html.findAll('span',{'itemprop':'dateCreated'})
card = np.array(page_html.findAll('div',{'class':'m-srp-card__desc flex__item'}))

# cleaning the data in card recordsto have only values and no empty values in the list
card_records = []
for j in range(len(card)):
    card_records.append([i for i in card[j].text.split('\n') if i!=''])
s2 = time.time()
print('Execution time:',s2-s1)

Execution time: 3.3217790126800537


### Extracting the data from the card_records variable

In [6]:
#How many records have the terms for Sale and for Sale in?
check = 0
for i in card_records:
    if 'for Sale' in i or 'for Sale in' in i:
        check+=1

In [7]:
#Now we can get the location info for each record since the next 2 values after for Sale in the list are location
location = []
for i in card_records:
    if 'for Sale' in i:
        location.append(i[i.index('for Sale')+1:i.index('for Sale')+3])
    elif 'for Sale in' in i:
        location.append(i[i.index('for Sale in')+1:i.index('for Sale in')+3])
location_series = pd.Series(location)
len(location)

390

In [8]:
def find_and_impute(records,search_term1,index_next,ifnot_term = None):
    location = []
    try:
        for i in records:
            if search_term1 in i:
                location.append(i[i.index(search_term1)+index_next])
            else:
                location.append(ifnot_term)
    except:
        location.append(np.nan)
    return location

In [9]:
# page_html

### Using the above created function to extract all avaiable features in card_records variable.

In [10]:
#Look for super area and carpet area. We will make manual changes later
area_calc_on = pd.Series(find_and_impute(records = card_records, search_term1 = 'carpet area',index_next = 0,ifnot_term = 'super area'))
#Extract the status of each property. If unavailable then imputing nan
status = pd.Series(find_and_impute(card_records,search_term1 = 'status',index_next = 1,ifnot_term = np.nan))
#Extracting the floor of each property. If unavailable then imputing nan
floors = pd.Series(find_and_impute(card_records,search_term1 = 'floor',index_next = 1,ifnot_term = np.nan))
#Extracting the transaction type for each property. New or resale.
new_or_resale = pd.Series(find_and_impute(card_records,search_term1 = 'transaction',index_next = 1,ifnot_term = np.nan))
# Extracting type of furnishing.
furnishing = pd.Series(find_and_impute(card_records,search_term1 = 'furnishing',index_next = 1,ifnot_term = np.nan))
#Extracting Society
society = pd.Series(find_and_impute(card_records,search_term1 = 'society',index_next = 1,ifnot_term = np.nan))
#Extracting number of bathrooms on property
bathrooms = pd.Series(find_and_impute(card_records,search_term1 = 'bathroom',index_next = 1,ifnot_term = np.nan))
#Extracting the builder name
advertiser_name = pd.Series([i[-1] for i in card_records])
#Extracting BHK
bhk = pd.Series([i[0] for i in card_records])
#Extracting the price from price_html
price = pd.Series([i.text for i in price_html])
#Extract parking
parking = pd.Series(find_and_impute(card_records,search_term1 = 'car parking',index_next = 1,ifnot_term = np.nan))
#Extract comments
comments = pd.Series(find_and_impute(card_records,search_term1 = 'read more',index_next = -1,ifnot_term = np.nan))

In [11]:
#Extract the date posted
date_txt = [i.text for i in date_created]
today = date.today()
yesterday = today - timedelta(days = 1) 
date_vals = []
for i in date_txt:
    if i.lower() == 'today':
        date_vals.append(today.strftime("%b")+" "+today.strftime("%d")+","+today.strftime(" '%y"))
    elif i.lower() == 'yesterday':
        date_vals.append(yesterday.strftime("%b")+" "+yesterday.strftime("%d")+","+yesterday.strftime(" '%y"))
    else:
        date_vals.append(i)
date_series = pd.Series(date_vals)

In [12]:
#Area values,facing,overlooking
#Find facing
facing = pd.Series(find_and_impute(card_records,search_term1 = 'facing',index_next = 1,ifnot_term = np.nan))
#Find overlooking
overlooking = pd.Series(find_and_impute(card_records,search_term1 = 'overlooking',index_next = 1,ifnot_term = np.nan))
#Area values
area_values = []
for i in card_records:
    if 'carpet area' in i:
        area_values.append(i[i.index('carpet area')+1])
    elif 'super area' in i:
        area_values.append(i[i.index('super area')+1])
area = pd.Series(area_values)
len(area)

390

In [13]:
df = pd.concat((area_calc_on,status,floors,new_or_resale,furnishing,society,
                bathrooms,advertiser_name,bhk,price,parking,comments,
                location_series,date_series,facing,overlooking,area),axis = 1)

n = ['Area_Type','Possession_Status','Floors','new_or_resale',
     'furnishing','society','bathrooms','advertiser_name','bhk',
     'price','parking','comments','location','date_of_posting',
     'facing','overlooking','area']

names = dict(zip(list(range(len(n))),n))
df_final = df.rename(names,axis = 1)

In [14]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Area_Type          390 non-null    object
 1   Possession_Status  386 non-null    object
 2   Floors             381 non-null    object
 3   new_or_resale      390 non-null    object
 4   furnishing         389 non-null    object
 5   society            206 non-null    object
 6   bathrooms          387 non-null    object
 7   advertiser_name    390 non-null    object
 8   bhk                390 non-null    object
 9   price              390 non-null    object
 10  parking            137 non-null    object
 11  comments           222 non-null    object
 12  location           390 non-null    object
 13  date_of_posting    390 non-null    object
 14  facing             295 non-null    object
 15  overlooking        275 non-null    object
 16  area               390 non-null    object
dt

In [15]:
#Save to csv file

In [16]:
if len(os.listdir('Data'))<=1:
    df_final.to_csv('Data/scrapped.csv',index = False)
else:
    new_data = df_final
    old_data = pd.read_csv('Data/scrappedv'+str(len(os.listdir('Data'))-1)+'.csv')
    new_final_data = pd.concat((new_data,old_data)).reset_index(drop = True)
    new_final_data.to_csv('Data/scrappedv'+str(len(os.listdir('Data')))+'.csv',index = False)

In [17]:
print(pd.read_csv('Data/scrappedv'+str(len(os.listdir('Data'))-1)+'.csv').shape)
pd.read_csv('Data/scrappedv'+str(len(os.listdir('Data'))-1)+'.csv').sample(2)

(2561, 17)


,Area_Type,Possession_Status,Floors,new_or_resale,furnishing,society,bathrooms,advertiser_name,bhk,price,parking,comments,location,date_of_posting,facing,overlooking,area
118,carpet area,Ready to Move,2 out of 11 floors,New Property,Unfurnished,Jyoti Breeze,2.0,View Agent Profile,1 BHK Apartment,₹ 48 Lac,1 Open,Venkatesh Jyoti Breeze is an premium Project l...,"['Jyoti Breeze,', 'Mira Road']","Feb 04, '21",East,"Garden/Park, Main Road",480 sqft
46,carpet area,Possession by Jan '22,5 out of 14 floors,New Property,Semi-Furnished,Agarwal Paramount,2.0,100+ Buyers Served,2 BHK Apartment,₹ 47 Lac,NaN,NaN,"['Agarwal Paramount,', 'Virar West']","Feb 05, '21",East,"Garden/Park, Main Road",604 sqft
